# Install & pin packages (Colab)

In [1]:
# A1) Minimal pins that don't fight Colab preinstalls
%pip -q install --upgrade pip setuptools wheel

# A2) Satisfy packages that complained earlier (numpy & jedi)
%pip -q install "numpy==2.0.2" "jedi>=0.19.1"

# A3) HF + Diffusers stack compatible with Colab & numpy 2.0.2
%pip -q install --upgrade --force-reinstall \
  "transformers==4.46.2" \
  "huggingface_hub==0.33.5" \
  "accelerate==0.31.0" \
  "diffusers==0.31.0" \
  "safetensors>=0.4.3" \
  "xformers>=0.0.27" \
  "pillow>=10.3.0"

# A4) Backend + Frontend
%pip -q install "fastapi==0.115.0" "uvicorn[standard]==0.30.6" "pyngrok==7.2.3" "nest_asyncio==1.6.0" "pydantic==2.9.2" "streamlit==1.38.0" "python-dotenv==1.0.1"

import sys, numpy, transformers, diffusers, huggingface_hub, fastapi, streamlit
print("Python:", sys.version.split()[0])
print("NumPy:", numpy.__version__)
print("transformers:", transformers.__version__)
print("diffusers:", diffusers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("FastAPI:", fastapi.__version__)
print("Streamlit:", streamlit.__version__)
print("✅ Environment ready")


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyarrow 15.0.2 requires numpy<2,>=1.16.6, but you have numpy 2.0.2 which is incompatible.
gradio 5.49.0 requires fastapi<1.0,>=0.115.2, but you have fastapi 0.115.0 which is incompatible.
gradio 5.49.0 requires starlette<1.0,>=0.40.0, but you have starlette 0.38.6 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyarrow 15.0.2 requires numpy<2,>=1.16.6, but you have numpy 2.3.3 which is incompatible.
streamlit 1.38.0 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
streamlit 1.38.0 requires pillow<11,>=7.1.0, but you have pillow 

# B) Set secrets safely for this session (no hardcoding)

In [3]:
# Paste NEWLY ROTATED keys when prompted (input is hidden)
import os, getpass

print("Enter secrets (they will not be printed):")
GOOGLE_API_KEY = getpass.getpass("Google API key: ")
OPENAI_API_KEY = getpass.getpass("OpenAI-style key: ")
APP_API_TOKEN  = getpass.getpass("App token (for Streamlit→FastAPI auth): ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["APP_API_TOKEN"]  = APP_API_TOKEN

print("GOOGLE_API_KEY set:", bool(os.environ.get("GOOGLE_API_KEY")))
print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))
print("APP_API_TOKEN set:",  bool(os.environ.get("APP_API_TOKEN")))


Enter secrets (they will not be printed):
Google API key: ··········
OpenAI-style key: ··········
App token (for Streamlit→FastAPI auth): ··········
GOOGLE_API_KEY set: True
OPENAI_API_KEY set: True
APP_API_TOKEN set: True


# C) Project layout + shared config

In [4]:
import json
from pathlib import Path

ROOT = Path("Week_7")
(APP := ROOT/"app").mkdir(parents=True, exist_ok=True)
(BACK := ROOT/"backend").mkdir(parents=True, exist_ok=True)
(ROOT/"diffusion").mkdir(parents=True, exist_ok=True)
(ROOT/"screenshots").mkdir(parents=True, exist_ok=True)
(ROOT/"lora_weights").mkdir(parents=True, exist_ok=True)

run_cfg = {
    "model_id": "runwayml/stable-diffusion-v1-5",
    "negative_prompt": "low quality, blurry, nsfw, text watermark, logo, distorted text",
    "sd_steps": 25,
    "sd_guidance": 7.5,
    "size": [512, 512],
    "guardrails": {"max_hops": 3, "unsafe_terms": ["nsfw", "violent", "illegal"]},
}
(ROOT/"week7_run_config.json").write_text(json.dumps(run_cfg, indent=2))
print("✅ Wrote:", (ROOT/"week7_run_config.json").resolve())


✅ Wrote: /content/Week_7/week7_run_config.json


# D) Backend settings (reads env keys)

In [5]:
%%writefile {BACK}/settings.py
import os

class Settings:
    google_api_key: str = os.environ.get("GOOGLE_API_KEY", "")
    openai_api_key: str = os.environ.get("OPENAI_API_KEY", "")
    app_api_token: str  = os.environ.get("APP_API_TOKEN", "")  # bearer auth for UI→API

settings = Settings()


Writing Week_7/backend/settings.py


# E) FastAPI backend: /qa, /generate, /agent, /health

In [6]:
%%writefile {BACK}/main.py
import time, os, uuid, json
from pathlib import Path
from typing import List, Optional, Dict
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
import torch
from PIL import Image
from .settings import settings

ROOT = Path(__file__).resolve().parents[1]
OUT_DIR = ROOT / "diffusion"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CFG = json.loads((ROOT/"week7_run_config.json").read_text())

MODEL_ID = CFG["model_id"]
NEGATIVE = CFG["negative_prompt"]
SD_STEPS = int(CFG["sd_steps"])
SD_GUIDE = float(CFG["sd_guidance"])
W, H = CFG["size"]
GUARD = CFG["guardrails"]

_sd_pipe = None
def get_sd():
    global _sd_pipe
    if _sd_pipe is None:
        from diffusers import StableDiffusionPipeline
        device = "cuda" if torch.cuda.is_available() else "cpu"
        _sd_pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID,
            torch_dtype=torch.float16 if device=="cuda" else torch.float32
        )
        if device == "cuda":
            _sd_pipe.enable_attention_slicing()
            _sd_pipe.to(device)
    return _sd_pipe

def guardrails(text: str):
    for bad in GUARD.get("unsafe_terms", []):
        if bad.lower() in text.lower():
            raise HTTPException(status_code=400, detail=f"Blocked by guardrails: {bad}")

def require_auth(authorization: str | None = Header(default=None)):
    token = settings.app_api_token
    if not token:  # auth disabled if not set
        return
    if not authorization or not authorization.startswith("Bearer "):
        raise HTTPException(status_code=401, detail="Missing bearer token")
    if authorization.split(" ", 1)[1].strip() != token:
        raise HTTPException(status_code=403, detail="Invalid token")

# ----- Schemas
class QARequest(BaseModel):
    query: str = Field(...)

class QAResponse(BaseModel):
    answer: str
    citations: List[Dict[str,str]]
    latency_ms: int

class GenRequest(BaseModel):
    prompt: str
    negative_prompt: Optional[str] = NEGATIVE
    steps: Optional[int] = SD_STEPS
    guidance: Optional[float] = SD_GUIDE
    height: Optional[int] = H
    width: Optional[int] = W

class GenResponse(BaseModel):
    filename: str
    latency_ms: int

class AgentRequest(BaseModel):
    input: str

class AgentHop(BaseModel):
    tool: str
    input: str
    output_preview: str
    latency_ms: int

class AgentResponse(BaseModel):
    final: str
    hops: List[AgentHop]
    citations: List[Dict[str,str]] = []
    image_filename: Optional[str] = None
    latency_ms: int

app = FastAPI(title="Week 7 Backend", version="1.0")

@app.get("/health")
def health():
    return {"ok": True, "model": MODEL_ID}

# ----- Stub QA (replace with your Graph-RAG/Multi-Hop)
def project_qa(query: str):
    # If you call an external API, read keys from `settings.openai_api_key` / `settings.google_api_key`.
    ans = f"Stubbed answer for: {query} (replace with your Graph-RAG/Multi-Hop)."
    cites = [
        {"title": "Week 7 Repo", "url": "https://github.com/sudhakarjerribanda/Capston_Handson/tree/main/Week_7"},
        {"title": "Task 1 Colab", "url": "https://colab.research.google.com/drive/1yZd5zUUmNjeDH-_mpVEkNCO4fonsmW69?usp=sharing"}
    ]
    return ans, cites

@app.post("/qa", response_model=QAResponse)
def qa(req: QARequest, _=require_auth()):
    t0 = time.time()
    guardrails(req.query)
    ans, cites = project_qa(req.query)
    return QAResponse(answer=ans, citations=cites, latency_ms=int((time.time()-t0)*1000))

@app.post("/generate", response_model=GenResponse)
def generate(req: GenRequest, _=require_auth()):
    t0 = time.time()
    guardrails(req.prompt)
    pipe = get_sd()
    img = pipe(
        prompt=req.prompt,
        negative_prompt=req.negative_prompt or NEGATIVE,
        num_inference_steps=req.steps or SD_STEPS,
        guidance_scale=req.guidance or SD_GUIDE,
        height=req.height or H,
        width=req.width or W
    ).images[0]
    fname = f"sd_{uuid.uuid4().hex[:8]}.png"
    img.save(OUT_DIR/fname)
    return GenResponse(filename=fname, latency_ms=int((time.time()-t0)*1000))

@app.post("/agent", response_model=AgentResponse)
def agent(req: AgentRequest, _=require_auth()):
    t0 = time.time()
    guardrails(req.input)
    hops = []
    keywords = ["image","diagram","illustration","infographic","generate","sd"]
    if any(k in req.input.lower() for k in keywords):
        g0 = time.time()
        prompt = req.input + ", flat vector, teal+indigo accents, clean typography, rounded cards, subtle shadows"
        pipe = get_sd()
        img = pipe(
            prompt=prompt, negative_prompt=NEGATIVE,
            num_inference_steps=SD_STEPS, guidance_scale=SD_GUIDE,
            height=H, width=W
        ).images[0]
        fname = f"sd_{uuid.uuid4().hex[:8]}.png"
        img.save(OUT_DIR/fname)
        hops.append(AgentHop(tool="stable_diffusion", input=prompt, output_preview=fname, latency_ms=int((time.time()-g0)*1000)))
        final = f"Generated image {fname} for: {req.input}"
        cites = []
        image = fname
    else:
        q0 = time.time()
        ans, cites = project_qa(req.input)
        hops.append(AgentHop(tool="project_qa", input=req.input, output_preview=ans[:120], latency_ms=int((time.time()-q0)*1000)))
        final = ans
        image = None

    # Append tiny metric
    try:
        mpath = ROOT/"metrics.json"
        rec = {"ts": int(time.time()), "input": req.input, "latency_ms": int((time.time()-t0)*1000),
               "hops": [h.model_dump() for h in hops], "image": image}
        data = json.loads(mpath.read_text()) if mpath.exists() else []
        data.append(rec); mpath.write_text(json.dumps(data, indent=2))
    except Exception:
        pass

    return AgentResponse(final=final, hops=hops, citations=cites, image_filename=image, latency_ms=int((time.time()-t0)*1000))


Writing Week_7/backend/main.py


# F) Start backend + expose with ngrok

In [12]:
# ✅ Start FastAPI (port 8000) and expose: ngrok if token, else cloudflared
import os, time, threading, re, subprocess, requests
from pathlib import Path
import nest_asyncio, uvicorn

# --- 1) Run the FastAPI backend on 8000 ---
nest_asyncio.apply()

def run_api():
    uvicorn.run("Week_7.backend.main:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_api, daemon=True)
thread.start()
time.sleep(2)  # give server a moment

# --- 2) Helpers to open a public URL ---
def _ensure_cloudflared():
    if not Path("cloudflared").exists():
        # Download static binary
        !curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared -#
        !chmod +x cloudflared

def _start_cloudflared(port: int):
    """Start a Cloudflare tunnel and return (public_url, process)."""
    _ensure_cloudflared()
    proc = subprocess.Popen(
        ["./cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}", "--no-autoupdate"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
    )
    public_url = None
    start = time.time()
    while time.time() - start < 30:
        line = proc.stdout.readline()
        if not line:
            time.sleep(0.1); continue
        m = re.search(r"https://[-\w]+\.trycloudflare\.com", line.strip())
        if m:
            public_url = m.group(0)
            break
    if not public_url:
        proc.terminate()
        raise RuntimeError("cloudflared did not produce a public URL in time.")
    return public_url, proc

def _start_ngrok(port: int):
    """Start ngrok if NGROK_AUTHTOKEN is set. Return (public_url, None)."""
    from pyngrok import ngrok, conf
    token = os.environ.get("NGROK_AUTHTOKEN", "").strip()
    if not token:
        raise RuntimeError("NGROK_AUTHTOKEN not set")
    conf.get_default().auth_token = token
    # Close any existing tunnels cleanly
    for t in ngrok.get_tunnels():
        try: ngrok.disconnect(t.public_url)
        except: pass
    url = ngrok.connect(addr=port, proto="http").public_url
    return url, None

def expose_port(port: int):
    """Try ngrok if token present; otherwise fallback to cloudflared."""
    if os.environ.get("NGROK_AUTHTOKEN", "").strip():
        url, proc = _start_ngrok(port)
        print(f"✅ ngrok tunnel ready on port {port}: {url}")
        return url, proc
    else:
        url, proc = _start_cloudflared(port)
        print(f"✅ cloudflared tunnel ready on port {port}: {url}")
        return url, proc

# --- 3) Expose backend and save URL for Streamlit ---
backend_url, backend_proc = expose_port(8000)
Path("BACKEND_URL.txt").write_text(backend_url)
print("Backend URL:", backend_url)
print("Health     :", f"{backend_url}/health")

# --- 4) Quick health check ---
try:
    print("Health JSON:", requests.get(f"{backend_url}/health", timeout=20).json())
except Exception as e:
    print("Health check failed:", e)


Exception in thread Thread-4 (run_api):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-2816236014.py", line 10, in run_api
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 26, in run
    loop = asyncio.get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 40, in _get_event_loop
    loop = events.get_event_loop_policy().get_event_loop()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr

✅ cloudflared tunnel ready on port 8000: https://missouri-complicated-chelsea-boat.trycloudflare.com
Backend URL: https://missouri-complicated-chelsea-boat.trycloudflare.com
Health     : https://missouri-complicated-chelsea-boat.trycloudflare.com/health
Health check failed: HTTPSConnectionPool(host='missouri-complicated-chelsea-boat.trycloudflare.com', port=443): Max retries exceeded with url: /health (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7ccb4d681610>: Failed to resolve 'missouri-complicated-chelsea-boat.trycloudflare.com' ([Errno -2] Name or service not known)"))


## G) Smoke tests (with bearer auth if set)

In [14]:
import os, requests, json, textwrap
from pathlib import Path

LOCAL = "http://127.0.0.1:8000/health"
PUBLIC = (Path("BACKEND_URL.txt").read_text().strip().rstrip("/")) + "/health"

def peek(url):
    try:
        r = requests.get(url, timeout=20)
        ct = r.headers.get("content-type", "")
        print(f"\nURL: {url}")
        print(f"Status: {r.status_code} | Content-Type: {ct}")
        body = r.text
        print("Body preview:")
        print(textwrap.shorten(body.replace("\n"," ")[:400], width=400, placeholder=" …"))
        return r
    except Exception as e:
        print(f"\nURL: {url} -> ERROR: {e}")
        return None

r_local = peek(LOCAL)      # should be JSON from FastAPI if backend is up
r_public = peek(PUBLIC)    # if this is HTML, it’s the tunnel page or an error page



URL: http://127.0.0.1:8000/health -> ERROR: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ccb4d9104d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

URL: https://missouri-complicated-chelsea-boat.trycloudflare.com/health
Status: 502 | Content-Type: text/plain; charset=utf-8
Body preview:
502 Bad Gateway Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared


In [15]:
import requests, json
from pathlib import Path

BASE = Path("BACKEND_URL.txt").read_text().strip().rstrip("/")
headers = {"Authorization": f"Bearer {os.environ['APP_API_TOKEN']}"} if os.environ.get("APP_API_TOKEN") else {}

def get_json(url, method="GET", **kwargs):
    try:
        r = requests.request(method, url, timeout=kwargs.pop("timeout", 30), **kwargs)
        ct = r.headers.get("content-type","")
        print(f"\n{method} {url} -> {r.status_code} {ct}")
        if "application/json" in ct.lower():
            data = r.json()
            print(json.dumps(data, indent=2)[:400], "...")
            return data
        else:
            # Not JSON – show a preview, don't raise
            print(r.text[:400], "...")
            return None
    except Exception as e:
        print(f"Request error: {e}")
        return None

# Health
get_json(f"{BASE}/health", "GET")

# QA
get_json(f"{BASE}/qa", "POST", json={"query":"Explain the IMRaD template in our medical text app"}, headers=headers, timeout=60)

# Generate
get_json(f"{BASE}/generate", "POST", json={"prompt":"infographic of a trust game with BDI panels, teal+indigo, flat vector, presentation-ready"}, headers=headers, timeout=180)



GET https://missouri-complicated-chelsea-boat.trycloudflare.com/health -> 502 text/plain; charset=utf-8
502 Bad Gateway
Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared
 ...

POST https://missouri-complicated-chelsea-boat.trycloudflare.com/qa -> 502 text/plain; charset=utf-8
502 Bad Gateway
Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared
 ...

POST https://missouri-complicated-chelsea-boat.trycloudflare.com/generate -> 502 text/plain; charset=utf-8
502 Bad Gateway
Unable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared
 ...


# H) Streamlit UI (writes Week_7/app/app.py)

In [16]:
%%writefile Week_7/app/app.py
import os, time, json, requests
from pathlib import Path
import streamlit as st

st.set_page_config(page_title="Week 7 App", page_icon="🧪", layout="wide")

# Resolve backend URL: secrets -> env -> file -> default local
BACKEND_URL    = st.secrets.get("BACKEND_URL", os.environ.get("BACKEND_URL", ""))
APP_API_TOKEN  = st.secrets.get("APP_API_TOKEN", os.environ.get("APP_API_TOKEN", ""))

if not BACKEND_URL and Path("BACKEND_URL.txt").exists():
    BACKEND_URL = Path("BACKEND_URL.txt").read_text().strip()

if not BACKEND_URL:
    BACKEND_URL = "http://127.0.0.1:8000"  # fallback

headers = {"Authorization": f"Bearer {APP_API_TOKEN}"} if APP_API_TOKEN else {}

st.sidebar.success("Backend: " + BACKEND_URL)
st.title("CS 5588 – Week 7 App (Sudhakar Reddy Jerribanda)")
tabs = st.tabs(["Ask (QA)", "Generate (SD)", "Agent"])

with tabs[0]:
    st.subheader("Ask your project model")
    q = st.text_input("Question", "How do we visualize trust (BDI) in the app?")
    if st.button("Run QA"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/qa", json={"query": q}, headers=headers, timeout=120)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                st.markdown("**Answer:**")
                st.write(data.get("answer",""))
                st.markdown("**Citations:**")
                for c in data.get("citations", []):
                    st.write(f"- [{c.get('title','link')}]({c.get('url','#')})")
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")

with tabs[1]:
    st.subheader("Generate a project-style visual (Stable Diffusion)")
    prompt = st.text_area("Prompt", "infographic of a trust game with BDI panels, teal+indigo, flat vector, presentation-ready", height=100)
    if st.button("Generate Image"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/generate", json={"prompt": prompt}, headers=headers, timeout=300)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                fname = data.get("filename")
                if fname:
                    st.image(str(Path("Week_7")/"diffusion"/fname), caption=fname, use_column_width=True)
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")

with tabs[2]:
    st.subheader("Agent Orchestration (Plan → Execute → Aggregate)")
    ipt = st.text_area("Ask or request a visual", "Generate a clean isometric diagram of recruiter → MDT → ICT triage flow")
    if st.button("Run Agent"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/agent", json={"input": ipt}, headers=headers, timeout=300)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                st.markdown("**Final:**")
                st.write(data.get("final",""))
                st.markdown("**Hops (trace):**")
                for h in data.get("hops", []):
                    st.code(json.dumps(h, indent=2))
                img = data.get("image_filename")
                if img:
                    st.image(str(Path("Week_7")/"diffusion"/img), caption=img, use_column_width=True)
                if data.get("citations"):
                    st.markdown("**Citations:**")
                    for c in data["citations"]:
                        st.write(f"- [{c.get('title','link')}]({c.get('url','#')})")
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")


Writing Week_7/app/app.py


## I) Launch Streamlit and expose (reuses expose_port; defines a fallback if missing)

In [19]:
# ---- I) Launch Streamlit and expose on 8501 ----
import os, threading, time, subprocess, re
from pathlib import Path

# If expose_port isn't defined (e.g., new session), define a minimal fallback (cloudflared only)
if "expose_port" not in globals():
    import subprocess, time, re
    def _ensure_cloudflared():
        if not Path("cloudflared").exists():
            !curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared -#
            !chmod +x cloudflared
    def expose_port(port: int):
        _ensure_cloudflared()
        proc = subprocess.Popen(
            ["./cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}", "--no-autoupdate"],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1
        )
        public_url = None; start = time.time()
        while time.time() - start < 30:
            line = proc.stdout.readline()
            if not line: time.sleep(0.1); continue
            m = re.search(r"https://[-\w]+\.trycloudflare\.com", line.strip())
            if m: public_url = m.group(0); break
        if not public_url:
            proc.terminate(); raise RuntimeError("cloudflared did not produce a public URL in time.")
        return public_url, proc

# Pass backend URL/token to the Streamlit process via env (so app.py can read them if needed)
BASE = (Path("BACKEND_URL.txt").read_text().strip() if Path("BACKEND_URL.txt").exists() else "http://127.0.0.1:8000")
os.environ["BACKEND_URL"]   = BASE
os.environ["APP_API_TOKEN"] = os.environ.get("APP_API_TOKEN","")

def run_streamlit():
    subprocess.Popen([
        "streamlit", "run", str(Path("Week_7")/"app"/"app.py"),
        "--server.address=0.0.0.0", "--server.port=8501"
    ])

threading.Thread(target=run_streamlit, daemon=True).start()
time.sleep(3)  # give Streamlit time to boot

frontend_url, frontend_proc = expose_port(8501)
print("✅ Backend:", BASE)
print("✅ Frontend:", frontend_url)


✅ cloudflared tunnel ready on port 8501: https://members-watts-soft-afternoon.trycloudflare.com
✅ Backend: https://missouri-complicated-chelsea-boat.trycloudflare.com
✅ Frontend: https://members-watts-soft-afternoon.trycloudflare.com


In [18]:
%%writefile Week_7/app/app.py
import os, json, requests
from pathlib import Path
import streamlit as st

st.set_page_config(page_title="Week 7 App", page_icon="🧪", layout="wide")

def safe_secret(key: str, default: str = "") -> str:
    # Prefer environment (works in Colab), then secrets if available.
    val = os.environ.get(key)
    if val:
        return val
    try:
        # Only try st.secrets if a secrets file exists
        # (Streamlit raises FileNotFoundError when no secrets.toml)
        _ = st.secrets  # access to trigger load; will raise if missing
        return st.secrets.get(key, default)
    except Exception:
        return default

# Resolve backend URL/token with robust fallback order:
# 1) ENV  2) secrets.toml (if present)  3) BACKEND_URL.txt  4) local default
BACKEND_URL   = safe_secret("BACKEND_URL", "")
APP_API_TOKEN = safe_secret("APP_API_TOKEN", "")

if not BACKEND_URL and Path("BACKEND_URL.txt").exists():
    BACKEND_URL = Path("BACKEND_URL.txt").read_text().strip()

if not BACKEND_URL:
    BACKEND_URL = "http://127.0.0.1:8000"

headers = {"Authorization": f"Bearer {APP_API_TOKEN}"} if APP_API_TOKEN else {}

st.sidebar.success("Backend: " + BACKEND_URL)
st.title("CS 5588 – Week 7 App (Sudhakar Reddy Jerribanda)")
tabs = st.tabs(["Ask (QA)", "Generate (SD)", "Agent"])

with tabs[0]:
    st.subheader("Ask your project model")
    q = st.text_input("Question", "How do we visualize trust (BDI) in the app?")
    if st.button("Run QA"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/qa", json={"query": q}, headers=headers, timeout=120)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                st.markdown("**Answer:**")
                st.write(data.get("answer",""))
                st.markdown("**Citations:**")
                for c in data.get("citations", []):
                    st.write(f"- [{c.get('title','link')}]({c.get('url','#')})")
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")

with tabs[1]:
    st.subheader("Generate a project-style visual (Stable Diffusion)")
    prompt = st.text_area("Prompt", "infographic of a trust game with BDI panels, teal+indigo, flat vector, presentation-ready", height=100)
    if st.button("Generate Image"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/generate", json={"prompt": prompt}, headers=headers, timeout=300)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                fname = data.get("filename")
                if fname:
                    st.image(str(Path("Week_7")/"diffusion"/fname), caption=fname, use_column_width=True)
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")

with tabs[2]:
    st.subheader("Agent Orchestration (Plan → Execute → Aggregate)")
    ipt = st.text_area("Ask or request a visual", "Generate a clean isometric diagram of recruiter → MDT → ICT triage flow")
    if st.button("Run Agent"):
        try:
            r = requests.post(f"{BACKEND_URL.rstrip('/')}/agent", json={"input": ipt}, headers=headers, timeout=300)
            ct = (r.headers.get("content-type") or "").lower()
            if "application/json" in ct:
                data = r.json()
                st.markdown("**Final:**")
                st.write(data.get("final",""))
                st.markdown("**Hops (trace):**")
                for h in data.get("hops", []):
                    st.code(json.dumps(h, indent=2))
                img = data.get("image_filename")
                if img:
                    st.image(str(Path("Week_7")/"diffusion"/img), caption=img, use_column_width=True)
                if data.get("citations"):
                    st.markdown("**Citations:**")
                    for c in data["citations"]:
                        st.write(f"- [{c.get('title','link')}]({c.get('url','#')})")
                st.caption(f"Latency: {data.get('latency_ms',0)} ms")
            else:
                st.warning("Non-JSON response from backend. Preview below:")
                st.code(r.text[:600])
        except Exception as e:
            st.error(f"Error: {e}")


Overwriting Week_7/app/app.py


In [20]:
# Optional: create a secrets.toml so Streamlit can read from it
from pathlib import Path
Path("/content/.streamlit").mkdir(parents=True, exist_ok=True)
Path("/content/.streamlit/secrets.toml").write_text(
    'BACKEND_URL = "{}"\nAPP_API_TOKEN = "{}"\n'.format(
        os.environ.get("BACKEND_URL","http://127.0.0.1:8000"),
        os.environ.get("APP_API_TOKEN","")
    )
)
print("Wrote /content/.streamlit/secrets.toml")


Wrote /content/.streamlit/secrets.toml
